# Homework

In [1]:
import dlt
import duckdb

#### Question 1

In [2]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

In [3]:
limit_1 = 5
generator_1 = square_root_generator(limit_1)
sum_of_outputs = sum(generator_1)
sum_of_outputs

8.382332347441762

#### Question 2 

In [4]:
limit_2 = 13
generator_2 = square_root_generator(limit_2)
thirteenth_number = next(x for i, x in enumerate(generator_2, 1) if i == 13)
thirteenth_number

3.605551275463989

#### Question 3

In [5]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)

{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


In [6]:
generator_pipeline = dlt.pipeline(
    destination="duckdb",
    dataset_name="question3"
)

info = generator_pipeline.run(
    data=people_1(),
    table_name="people",
    write_disposition="replace"
)
info = generator_pipeline.run(
    data=people_2(),
    table_name="people",
    write_disposition="append"
)

In [7]:
conn = duckdb.connect(database=f"{generator_pipeline.pipeline_name}.duckdb")

In [8]:
conn.sql(f"SET search_path = '{generator_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("SHOW TABLES"))

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people              │
└─────────────────────┘

In [9]:
conn.sql("SELECT * FROM people").df()

,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707256196.52781,rdrgo+R8hrSAsg,None
1,2,Person_2,27,City_A,1707256196.52781,8w2Qm1yEYSUHYg,None
2,3,Person_3,28,City_A,1707256196.52781,n+3IpHbbPXsS1A,None
3,4,Person_4,29,City_A,1707256196.52781,KIYRYi7Hp+RFHQ,None
4,5,Person_5,30,City_A,1707256196.52781,aGCjgkl7ddWLxA,None
5,3,Person_3,33,City_B,1707256197.080097,eErWh3hhYDIzog,Job_3
6,4,Person_4,34,City_B,1707256197.080097,eKJmn8PlunX7sA,Job_4
7,5,Person_5,35,City_B,1707256197.080097,GnPhniAsQaMeww,Job_5
8,6,Person_6,36,City_B,1707256197.080097,Nj7+PJOM9tdxXQ,Job_6
9,7,Person_7,37,City_B,1707256197.080097,vJfVTDv1YE/xYQ,Job_7


In [10]:
conn.sql("SELECT SUM(age) FROM people").df()

,sum(age)
0,353.0


#### Question 4

In [14]:
pipeline = dlt.pipeline(
    destination="duckdb",
    dataset_name="question4"
)

info = pipeline.run(
    data=people_1(),
    table_name="people",
    write_disposition="replace",
    primary_key="ID"
)

In [15]:
conn = duckdb.connect(database=f"{pipeline.pipeline_name}.duckdb")
conn.sql(f"SET search_path = '{pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("SHOW TABLES"))

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people              │
└─────────────────────┘

In [16]:
conn.sql("SELECT * FROM people").df()

,id,name,age,city,_dlt_load_id,_dlt_id
0,1,Person_1,26,City_A,1707256364.2091234,F+TmhRoGztwDEQ
1,2,Person_2,27,City_A,1707256364.2091234,cm1xJ+bYEfHXTQ
2,3,Person_3,28,City_A,1707256364.2091234,GvMMMPnCNOodSQ
3,4,Person_4,29,City_A,1707256364.2091234,in2V274zubRShw
4,5,Person_5,30,City_A,1707256364.2091234,Q2aGKUZgOfkISQ


In [17]:
info = pipeline.run(
    data=people_2(),
    table_name="people",
    write_disposition="merge",
    primary_key="ID"
)

In [18]:
conn.sql("SELECT * FROM people ORDER BY id").df()

,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707256364.2091234,F+TmhRoGztwDEQ,None
1,2,Person_2,27,City_A,1707256364.2091234,cm1xJ+bYEfHXTQ,None
2,3,Person_3,33,City_B,1707256370.805916,QjQuGq+GhO7gng,Job_3
3,4,Person_4,34,City_B,1707256370.805916,ECvyXY5DUqsSGg,Job_4
4,5,Person_5,35,City_B,1707256370.805916,1cIdpUdUaleLBw,Job_5
5,6,Person_6,36,City_B,1707256370.805916,kviGVm0VH6jSVg,Job_6
6,7,Person_7,37,City_B,1707256370.805916,aZSM0dcFmmhD8Q,Job_7
7,8,Person_8,38,City_B,1707256370.805916,AcEI0zp08HkfAA,Job_8


In [19]:
conn.sql("SELECT COUNT(*) FROM people").df()

,count_star()
0,8


In [20]:
conn.sql("SELECT SUM(age) FROM people").df()

,sum(age)
0,266.0
